<a href="https://colab.research.google.com/github/asyraffatha/Task-MachineLearning/blob/main/Week%2012/Fashion_Mnist_Asyraff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Load dataset
file_path = '/content/drive/MyDrive/week 12/fashion-mnist_test.csv'
data = pd.read_csv(file_path)

In [4]:
# Prepare data
X = data.iloc[:, 1:].values.reshape(-1, 1, 28, 28).astype(np.float32) / 255.0
y = data.iloc[:, 0].values


In [5]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Convert to PyTorch tensors
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [7]:
# CNN Model
class CNN(nn.Module):
    def __init__(self, kernel_size, pooling_type):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=kernel_size // 2)
        self.pool1 = nn.MaxPool2d(2) if pooling_type == 'max' else nn.AvgPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=kernel_size // 2)
        self.pool2 = nn.MaxPool2d(2) if pooling_type == 'max' else nn.AvgPool2d(2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
# Training function
def train_model(model, optimizer, scheduler, criterion, train_loader, test_loader, epochs, early_stop_patience):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    best_acc = 0
    patience = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        scheduler.step()

        # Evaluate
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()

        acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss/len(train_loader):.4f}, Accuracy: {acc:.4f}")

        # Early stopping
        if acc > best_acc:
            best_acc = acc
            patience = 0
        else:
            patience += 1

        if patience >= early_stop_patience:
            print("Early stopping...")
            break


In [9]:
# Experiments
kernel_sizes = [3, 5, 7]
pooling_types = ['max', 'avg']
epochs_list = [5, 50, 100, 250, 350]
optimizers = {'SGD': optim.SGD, 'RMSProp': optim.RMSprop, 'Adam': optim.Adam}

results = []

for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for optimizer_name, optimizer_class in optimizers.items():
            print(f"\nTesting with kernel size: {kernel_size}, pooling: {pooling_type}, optimizer: {optimizer_name}")

            model = CNN(kernel_size=kernel_size, pooling_type=pooling_type)
            optimizer = optimizer_class(model.parameters(), lr=0.001)
            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
            criterion = nn.CrossEntropyLoss()

            train_model(
                model, optimizer, scheduler, criterion,
                train_loader, test_loader,
                epochs=50,  # Change for specific experiments
                early_stop_patience=10
            )



Testing with kernel size: 3, pooling: max, optimizer: SGD
Epoch 1/50, Loss: 2.2989, Accuracy: 0.0990
Epoch 2/50, Loss: 2.2900, Accuracy: 0.1615
Epoch 3/50, Loss: 2.2810, Accuracy: 0.1915
Epoch 4/50, Loss: 2.2714, Accuracy: 0.2055
Epoch 5/50, Loss: 2.2606, Accuracy: 0.2515
Epoch 6/50, Loss: 2.2478, Accuracy: 0.3295
Epoch 7/50, Loss: 2.2318, Accuracy: 0.3775
Epoch 8/50, Loss: 2.2115, Accuracy: 0.4245
Epoch 9/50, Loss: 2.1857, Accuracy: 0.4735
Epoch 10/50, Loss: 2.1522, Accuracy: 0.4795
Epoch 11/50, Loss: 2.1296, Accuracy: 0.4775
Epoch 12/50, Loss: 2.1250, Accuracy: 0.4760
Epoch 13/50, Loss: 2.1203, Accuracy: 0.4715
Epoch 14/50, Loss: 2.1155, Accuracy: 0.4700
Epoch 15/50, Loss: 2.1106, Accuracy: 0.4705
Epoch 16/50, Loss: 2.1055, Accuracy: 0.4690
Epoch 17/50, Loss: 2.1003, Accuracy: 0.4670
Epoch 18/50, Loss: 2.0950, Accuracy: 0.4645
Epoch 19/50, Loss: 2.0895, Accuracy: 0.4630
Epoch 20/50, Loss: 2.0839, Accuracy: 0.4615
Early stopping...

Testing with kernel size: 3, pooling: max, optimize